In [1]:
import sys
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pathlib import Path

load_dotenv(override=True)

OPENAI_API_KEY = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]
MONGO_URI = os.environ["MONGO_URI"]
EMBEDDING_MODEL_NAME = os.environ["EMBEDDING_MODEL_NAME"]
EMBEDDING_DIMENSIONS = os.environ["EMBEDDING_DIMENSIONS"]
CHAT_MODEL_NAME = os.environ["CHAT_MODEL_NAME"]
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_CHATBOT_TEST_KEY_INTERNAL"]

DB_NAME = "gaia"
COLLECTION_NAME = "documents"
ATLAS_VECTOR_SEARCH_INDEX_NAME = "vector_index"
MAX_CHUNKS_TO_RETRIEVE=10
CHUNK_MIN_RELEVANCE_SCORE=0.4

MAX_TOKENS_FOR_RESPONSE = 1000
CHAT_MODEL_TEMPERATURE=0
CHAT_MODEL_FREQ_PENALTY=0.2
CHAT_MODEL_PRES_PENALTY=0.2
SHOW_VERBOSE=False


PARENT_PATH = Path.cwd().parent
EVA_SETTINGS_PATH = PARENT_PATH / 'evasettings'
EVA_SETTINGS_ENVIRONMENT_DIRECTORY = 'local'

In [2]:
models_path = PARENT_PATH / 'scripts' / 'models'
vectordatabases_path = PARENT_PATH / 'scripts' / 'vectordatabases'

if str(models_path) not in sys.path:
    sys.path.append(str(models_path))
if str(vectordatabases_path) not in sys.path:
    sys.path.append(str(vectordatabases_path))

from models import model_rag
from vectordatabases import BaseDB

In [3]:
# from ragas import evaluate
# from datasets import Dataset
# from ragas.metrics import (
#     answer_relevancy,
#     answer_correctness
# )

# metrics = [
#     answer_relevancy,
#     answer_correctness
# ]

# retrieved_contexts = [doc.page_content for doc in sources_documents]
# reference = " ".join(retrieved_contexts) if retrieved_contexts else ""
# response_dataset = [
#     {
#         "question": f"{self.summarized_history}\n{self.chat_data.user_input}",
#         "answer": response_text,
#         "retrieved_contexts": retrieved_contexts, 
#         "reference": reference 
#     }
# ]
# response_scores = self._run_ragas_evaluation(response_dataset)
# print("Generated Response Evaluation Scores: ", response_scores)

In [4]:
from langchain.vectorstores import MongoDBAtlasVectorSearch
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.runnables import RunnableSequence
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.schema import HumanMessage, SystemMessage

class RAG:
    def __init__(self, chat_data):
        self.chat_data = chat_data

    ## Public Methods
    def get_response(self):
        self.llm_eva = ChatOpenAI(
            model_name=self.chat_data.rag_settings.chat_model_name,
            temperature=self.chat_data.rag_settings.temperature,
            max_tokens=self.chat_data.rag_settings.max_tokens_for_response,
            openai_api_key=self.chat_data.llm_settings.llm_key
        )

        self.summarized_history = ""
        self.memory = None
        if self.chat_data.chat_history:
            self.summarized_history, self.memory = self._summarize_history()
        
        # Detect the intent first
        intent_name = self._detect_intent()
        print("Detected Intent: ", intent_name)
        print()

        # Now invoke the QA model to get the response
        qa = self._get_qa_instance(intent_name)
        result = qa.invoke({"question": self.chat_data.user_input})
        
        response_text = result.get("answer", "")
        response_text, display_source = self._format_response(response_text)
        if display_source is True:
            sources_documents = result.get("source_documents", [])            
            sources_list = self._extract_sources(sources_documents)
        else:
            sources_list=[]
        
        return model_rag.ChatResponse(response=response_text, sources=sources_list)

        
    ## Private Methods

    def _extract_sources(self, sources_documents):
        sources_list = []
        for doc in sources_documents:
            language=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "language"), ""), "")
            if language.lower()=="english":
                sources_list.append(
                    model_rag.Source(
                        source=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "source"), ""), ""),
                        type=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "type"), ""), ""),
                        title=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "title"), ""), ""),
                        country=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "country"), ""), ""),
                        language=doc.metadata.get(next((key for key in doc.metadata if key.lower() == "language"), ""), "")
                    )
                )
        return sources_list

    def _format_response(self, response_text):
        display_source = False
        try:
            if "<displaysource>" in response_text and "</displaysource>" in response_text:
                display_source = response_text.split("<displaysource>")[1].split("</displaysource>")[0].strip() == "true"
                response_text = response_text.replace(f"<displaysource>{'true' if display_source else 'false'}</displaysource>", "")
        except Exception as e:
            pass 
    
        return response_text, display_source
        
    
    def _load_template(self, project_template_directory_name, template_file_name):
        project_template_directory_path = os.path.join(EVA_SETTINGS_PATH, project_template_directory_name, EVA_SETTINGS_ENVIRONMENT_DIRECTORY)
        template_file_path = project_template_directory_path + '/' + template_file_name
        with open(template_file_path, "r") as file:
            return file.read()

    def _build_intent_detection_prompt(self):
        # Load intent detection template
        intent_detection_template = self._load_template(
            self.chat_data.prompt_template_directory_name, 
            self.chat_data.intent_detection_prompt_template_file_name
        )
        
        # Dynamically build the intent list from intent details
        intent_list = "\n".join(
            [f'- "{intent_name}": {intent_data.description}' for intent_name, intent_data in self.chat_data.intent_details.items()]
        )
        
        # Build the full prompt for intent detection
        prompt = intent_detection_template.format(
            user_input=self.chat_data.user_input,
            history=self.summarized_history,  # Use summarized history here
            intent_list=intent_list
        )
        return prompt

    def _build_chat_prompt(self, intent_name):
        # Load base template
        base_template = self._load_template(
            self.chat_data.prompt_template_directory_name, 
            self.chat_data.base_prompt_template_file_name
        )
        
        # Load intent-specific template or default to generic message
        if intent_name == "none":
            intent_template = ""
        else:
            intent_filename = self.chat_data.intent_details.get(intent_name).filename
            intent_template = self._load_template(self.chat_data.prompt_template_directory_name, intent_filename)
        
        # Build the full prompt using the base and intent templates
        return base_template.format(
            subinstructions=intent_template,
            history=self.summarized_history,  # Pass the summarized history for context
            summaries="{summaries}",
            question="{question}"
        )

    def _get_qa_retriever(self):
        llm_embeddings = OpenAIEmbeddings(
            model=self.chat_data.llm_settings.embedding_model_name,
            openai_api_key=self.chat_data.llm_settings.llm_key
        )
    
        db_instance = BaseDB().get_vector_db(
            self.chat_data.db_type,
            self.chat_data.db_settings,
            llm_embeddings
        )
        vector_store = db_instance.vector_index
    
        qa_retriever = vector_store.as_retriever(
            # search_type="similarity_score_threshold", 
            # search_kwargs={
            #     "k": self.chat_data.rag_settings.max_chunks_to_retrieve.value, 
            #     "score_threshold": self.chat_data.rag_settings.retrieved_chunks_min_relevance_score.value
            # }
            search_type="mmr",
            search_kwargs={"k": self.chat_data.rag_settings.max_chunks_to_retrieve.value, 
                           "fetch_k": 50, 
                           "lambda_mult": 0.5
            }
        )
        
        return qa_retriever


    def _get_qa_instance(self, intent_name):
        dynamic_prompt_content = self._build_chat_prompt(intent_name)
            
        prompt_template = PromptTemplate(
            template=dynamic_prompt_content,
            input_variables=['summaries', 'question']
        )
    
        qa_retriever = self._get_qa_retriever()

        if self.memory:
            chain_type_kwargs = {
                "verbose": SHOW_VERBOSE,
                "prompt": prompt_template,
                "memory": self.memory  # Include memory if available
            }
        else:
            chain_type_kwargs = {
                "verbose": SHOW_VERBOSE,
                "prompt": prompt_template
            }

        qa = RetrievalQAWithSourcesChain.from_chain_type(
            llm=self.llm_eva,
            chain_type="stuff",
            retriever=qa_retriever,
            return_source_documents=True,
            chain_type_kwargs=chain_type_kwargs
        )
    
        return qa


    def _detect_intent(self):
        intent_prompt = self._build_intent_detection_prompt()
        
        prompt_template = PromptTemplate(
            template=intent_prompt,
            input_variables=["user_input", "history", "intent_list"]
        )
        
        intent_chain = RunnableSequence(prompt_template, self.llm_eva)
        intent_result = intent_chain.invoke({
            "user_input": self.chat_data.user_input,  
            "history": self.summarized_history,  
            "intent_list": "\n".join([f'- "{intent_name}"' for intent_name in self.chat_data.intent_details.keys()])
        })

        detected_intent = intent_result.content.strip().strip(' "\'').lower()
        if detected_intent not in [intent_name.lower() for intent_name in self.chat_data.intent_details.keys()]:
            return "none"
        return detected_intent

    def _summarize_history(self):
        if not self.chat_data.chat_history:
            return "", None
        
        history = ChatMessageHistory()
        for conv in self.chat_data.chat_history:
            if conv.role.lower() == 'human':
                history.add_message(HumanMessage(content=conv.message))
            elif conv.role.lower() == 'ai':
                history.add_message(SystemMessage(content=conv.message))
            
        memory = ConversationSummaryMemory.from_messages(
            llm=self.llm_eva,
            chat_memory=history,
            return_messages=True,
            memory_key="history",
            input_key="question"
        )

        summarized_history = memory.buffer
        return summarized_history, memory



In [5]:
conversation_history = []


def get_chatbot_response(payload: model_rag.ChatRequest):
    chat_processor = RAG(payload)
    response = chat_processor.get_response()
    return response
   


def call_chatbot_endpoint(user_input_text):
    global conversation_history
    
    # Directly create an instance of model_rag.ChatRequest with the required values
    chat_request = model_rag.ChatRequest(
        db_type="mongodb",
        db_settings={
            "uri": MONGO_URI,  
            "db_name": DB_NAME,  
            "collection_name": COLLECTION_NAME,  
            "vector_index_name": ATLAS_VECTOR_SEARCH_INDEX_NAME,  
            "vector_similarity_function": "cosine"
        },
        llm_settings={
            "llm_key": OPENAI_API_KEY,  
            "vector_dimension_size": EMBEDDING_DIMENSIONS,  
            "embedding_model_name": EMBEDDING_MODEL_NAME
        },
        rag_settings={
            "chat_model_name": CHAT_MODEL_NAME,  
            "max_chunks_to_retrieve": MAX_CHUNKS_TO_RETRIEVE,  
            "retrieved_chunks_min_relevance_score": CHUNK_MIN_RELEVANCE_SCORE,
            "max_tokens_for_response": MAX_TOKENS_FOR_RESPONSE,  
            "temperature": CHAT_MODEL_TEMPERATURE,  
            "frequency_penalty": CHAT_MODEL_FREQ_PENALTY,
            "presence_penalty": CHAT_MODEL_PRES_PENALTY
        },
        user_input=user_input_text,
        chat_history=conversation_history,
        prompt_template_directory_name="gaia",  
        base_prompt_template_file_name="base_template.txt", 
        intent_detection_prompt_template_file_name="detect_intent.txt", 
        intent_details = {
            "diagnosis": {
                "filename": "diagnosis.txt",
                "description": "This intent covers queries related to diagnosing pests or problems affecting crops, including identifying potential pests or diseases based on symptoms, crop type, and location."
            },
            "symptoms_identification": {
                "filename": "symptoms_identification.txt",
                "description": "This intent provides detailed information about symptoms caused by a specific pest or problem, including visual indicators and progression of the symptoms."
            },
            "pest_list": {
                "filename": "pest_list.txt",
                "description": "This intent provides a list of pests that affect a specific crop in a specific country or region."
            },
            "ipm_pest_management": {
                "filename": "ipm_pest_management.txt",
                "description": "This intent provides integrated pest management (IPM) advice, including prevention strategies, biocontrol recommendations, and chemical pesticide usage for managing pests or diseases on crops."
            },
            "chemical_handling_safety": {
                "filename": "chemical_handling_safety.txt",
                "description": "This intent provides safety advice for handling and applying specific chemicals, including personal protective equipment (PPE), safe storage, and disposal recommendations."
            },
            "invasive_pest_status": {
                "filename": "invasive_pest_status.txt",
                "description": "This intent provides information on the current status, distribution, and spread of invasive pests in a specific country or region."
            },
            "dosage_recommendations": {
                "filename": "dosage_recommendations.txt",
                "description": "This intent provides dosage recommendations for chemical or biocontrol products, including application rates, frequency, and any location-specific restrictions or precautions."
            }
        }

    )
    
    chatbot_response = get_chatbot_response(payload=chat_request)
    
    conversation_history.append({
        "role": "Human",
        "message": user_input_text,  
    })
    conversation_history.append({
        "role": "AI",  
        "message": chatbot_response.response  
    })


    print('Bot''s Response:', chatbot_response.response)
    print('Sources: ', chatbot_response.sources)


In [6]:
# call_chatbot_endpoint("sdasd")

In [7]:
call_chatbot_endpoint("Hi, I need help to diagnose a problem related to coffee in Kenya?")

Detected Intent:  diagnosis

Bots Response: Sure, I can help with that! 

I understand that you're dealing with coffee in Kenya. Could you please provide more details about the symptoms you're observing and which part of the plant is affected? This will help me give you a more accurate diagnosis and recommendations. 

For example:
- Are you noticing any discoloration, wilting, or unusual spots on the leaves?
- Is the issue affecting the berries, leaves, stems, or roots?

Once I have more information, I can assist you better.
Sources:  []


In [8]:
call_chatbot_endpoint("It is affecting the leaves. Yellow and wilting leaves.")

Detected Intent:  diagnosis

Bots Response: I understand that you're dealing with coffee in Kenya, with symptoms such as yellowing and wilting leaves affecting the leaves.

Based on the information provided, it is possible that your coffee plants are suffering from Coffee Wilt Disease. Here are some details about this disease:

- **Cause**: Coffee Wilt Disease is caused by a fungus that lives in the soil and dead infected plant materials. The fungus enters the plant through wounds on the stem and roots.
- **Symptoms**: The fungus grows inside the plant and blocks the movement of water up the plant, leading to wilting leaves.
- **Management**: Unfortunately, there is no chemical control for Coffee Wilt Disease once the plant is infected. Preventative measures include:
  - Avoiding injuries to the plants that can serve as entry points for the fungus.
  - Removing and destroying infected plant materials to reduce the spread of the fungus.
  - Implementing good sanitation practices in the 